In [28]:
def weight_varible(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1,shape = shape)
    return tf.Variable(initial)
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding = 'SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize = [1,2,2,1],strides=[1,2,2,1],padding = 'SAME')

def identity_block(X_input, kernel_size, in_filter, out_filters, stage, block):
        # defining name basis
        block_name = 'res' + str(stage) + block
        f1, f2, f3 = out_filters
        with tf.variable_scope(block_name):
            X_shortcut = X_input

            #first
            W_conv1 = weight_varible([1, 1, in_filter, f1])
            X = tf.nn.conv2d(X_input, W_conv1, strides=[1, 1, 1, 1], padding='SAME')
            b_conv1 = bias_variable([f1])
            X = tf.nn.relu(X+ b_conv1)

            #second
            W_conv2 = weight_varible([kernel_size, kernel_size, f1, f2])
            X = tf.nn.conv2d(X, W_conv2, strides=[1, 1, 1, 1], padding='SAME')
            b_conv2 = bias_variable([f2])
            X = tf.nn.relu(X+ b_conv2)

            #third

            W_conv3 = weight_varible([1, 1, f2, f3])
            X = tf.nn.conv2d(X, W_conv3, strides=[1, 1, 1, 1], padding='SAME')
            b_conv3 = bias_variable([f3])
            X = tf.nn.relu(X+ b_conv3)
            #final step
            add = tf.add(X, X_shortcut)
            b_conv_fin = bias_variable([f3])
            add_result = tf.nn.relu(add+b_conv_fin)

        return add_result


#这里定义conv_block模块，由于该模块定义时输入和输出尺度不同，故需要进行卷积操作来改变尺度，从而得以相加
def convolutional_block( X_input, kernel_size, in_filter,
                            out_filters, stage, block, stride=2):
        # defining name basis
        block_name = 'res' + str(stage) + block
        with tf.variable_scope(block_name):
            f1, f2, f3 = out_filters

            x_shortcut = X_input
            #first
            W_conv1 = weight_varible([1, 1, in_filter, f1])
            X = tf.nn.conv2d(X_input, W_conv1,strides=[1, stride, stride, 1],padding='SAME')
            b_conv1 = bias_variable([f1])
            X = tf.nn.relu(X + b_conv1)

            #second
            W_conv2 =weight_varible([kernel_size, kernel_size, f1, f2])
            X = tf.nn.conv2d(X, W_conv2, strides=[1,1,1,1], padding='SAME')
            b_conv2 = bias_variable([f2])
            X = tf.nn.relu(X+b_conv2)

            #third
            W_conv3 = weight_varible([1,1, f2,f3])
            X = tf.nn.conv2d(X, W_conv3, strides=[1, 1, 1,1], padding='SAME')
            b_conv3 = bias_variable([f3])
            X = tf.nn.relu(X+b_conv3)
            #shortcut path
            W_shortcut =weight_varible([1, 1, in_filter, f3])
            x_shortcut = tf.nn.conv2d(x_shortcut, W_shortcut, strides=[1, stride, stride, 1], padding='VALID')

            #final
            add = tf.add(x_shortcut, X)
            #建立最后融合的权重
            b_conv_fin = bias_variable([f3])
            add_result = tf.nn.relu(add+ b_conv_fin)


        return add_result


In [ ]:
xs=tf.placeholder(tf.float32,[None,24,24,3])
ys=tf.placeholder(tf.float32,[None,88])
keep_prob = tf.placeholder(tf.float32)
keep_hide = tf.placeholder(tf.float32)
# lr1 = tf.Variable(0.0001, dtype=tf.float32)  
global_step = tf.Variable(0, dtype=tf.int32)
# lr2 = tf.Variable(0.00001, dtype=tf.float32)
# lr3 = tf.Variable(0.000005, dtype=tf.float32)  
lr = 0.0001
# learing_rate = tf.train.exponential_decay(0.001,120,10,0.96,staircase=False)
x_image = tf.reshape(xs,[-1,24,24,3])
keep_conv = 0.8
keep_hiden = 0.5

# #conv0
# W_conv0 = weight_varible([5,5,3,16])
# b_conv0 = bias_variable([16])
# h_conv0 = tf.nn.relu(conv2d(x_image,W_conv0)+b_conv0)
# fc_mean,fc_var = tf.nn.moments(h_conv0,axes = [0])
# scale = tf.Variable(tf.ones([16]))
# shift = tf.Variable(tf.zeros([16]))
# # h_LRN_conv1 = tf.nn.local_response_normalization(h_conv1,1,0,1,1)
# # h_bn_conv1 = tf.nn.batch_normalization(h_conv1,fc_mean,fc_var,shift,scale,0.001)
# h_pool0 = max_pool_2x2(h_conv0)
# h_dropout0 = tf.nn.dropout(h_pool0, keep_conv)

# conv1 layer
W_conv1 = weight_varible([5,5,3,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
fc_mean,fc_var = tf.nn.moments(h_conv1,axes = [0])
scale = tf.Variable(tf.ones([32]))
shift = tf.Variable(tf.zeros([32]))
# h_LRN_conv1 = tf.nn.local_response_normalization(h_conv1,1,0,1,1)
h_bn_conv1 = tf.nn.batch_normalization(h_conv1,fc_mean,fc_var,shift,scale,0.001)
h_pool1 = max_pool_2x2(h_bn_conv1)
h_dropout1 = tf.nn.dropout(h_pool1, keep_conv)

#stage 2
x = convolutional_block(X_input=h_pool1, kernel_size=3, in_filter=32,  out_filters=[32, 32, 128], stage=2, block='a', stride=1)
#上述conv_block操作后，尺寸变为14x14x256
x = identity_block(x, 3, 128, [32, 32, 128], stage=2, block='b' )
x = identity_block(x, 3, 128, [32, 32, 128], stage=2, block='c')
#上述操作后张量尺寸变成14x14x256
x = tf.nn.max_pool(x, [1, 2, 2, 1], strides=[1,2,2,1], padding='SAME')


# #stage 3
# x = convolutional_block(x, 3, 64, [64,64,128], 3, 'a')
# x = identity_block(x, 3, 128, [64,64,128], 3, 'b')
# x = identity_block(x, 3, 128, [64,64,128], 3, 'c')
# x = identity_block(x, 3, 128, [64,64,128], 3, 'd')

# #stage 4
# x = self.convolutional_block(x, 3, 512, [256, 256, 1024], 4, 'a')
# x = self.identity_block(x, 3, 1024, [256, 256, 1024], 4, 'b')
# x = self.identity_block(x, 3, 1024, [256, 256, 1024], 4, 'c')
# x = self.identity_block(x, 3, 1024, [256, 256, 1024], 4, 'd')
# x = self.identity_block (x, 3, 1024, [256, 256, 1024], 4, 'e')
# x = self.identity_block(x, 3, 1024, [256, 256, 1024], 4, 'f')

# #stage 5
# x = self.convolutional_block(x, 3, 1024, [512, 512, 2048], 5, 'a')
# x = self.identity_block(x, 3, 2048, [512, 512, 2048], 5, 'b')
# x = self.identity_block(x, 3, 2048, [512, 512, 2048], 5, 'c')
# func1 layer
nt_hpool = tf.contrib.layers.avg_pool2d(inputs=x,kernel_size=6,stride=6,padding='SAME')          #输出为[-1,1,1,64]
nt_hpool_flat = tf.reshape(nt_hpool,[-1,128])
# fun = tf.nn.dropout(nt_hpool_flat, keep_hiden)
# logits = tf.layers.dense(nt_hpool_flat, units=88, activation=tf.nn.softmax)
# # func2 layer
prediction = tf.contrib.layers.fully_connected(inputs=nt_hpool_flat,num_outputs=88,activation_fn=tf.nn.softmax)
predicted = tf.argmax(logits,1)

with tf.name_scope('loss'):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction),axis = 1))
    scalar_loss = tf.summary.scalar('loss', cross_entropy)

train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy,global_step)

# train_step1 = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
# train_step2 = tf.train.MomentumOptimizer(learning_rate=0.01,momentum=0.9).minimize(cross_entropy)
# train_step2 = tf.train.AdamOptimizer(lr).minimize(cross_entropy)

int_train_labels=train_sign.astype(int)
labels_a = np.eye(88,dtype=int)[int_train_labels]
int_test_labels=test_sign.astype(int)
labels_b = np.eye(88,dtype=int)[int_test_labels]
# x_batch, y_batch = get_Batch(images_train, labels_a, 16)
# sess = tf.Session()
# merged = tf.summary.merge_all()
# writer = tf.summary.FileWriter("C:\\Users\\lenovo\\Desktop\\CECS\\4528\\Final_project", sess.graph)
# sess.run(tf.global_variables_initializer())

def compute_accuracy(v_xs,v_ys):
    global logits
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1),tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,dtype=tf.float32))
    result = sess.run(accuracy,feed_dict={xs:v_xs,ys:v_ys, keep_prob: 1})
    return result

# correct_prediction = tf.equal(tf.argmax(logits,1),tf.argmax(ys,1))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction,dtype=tf.float32))
## steps
# for i in range(11):
#         int_train_labels=labels_train.astype(int)
#         labels_a = np.eye(11,dtype=int)[int_train_labels]
#         int_test_labels=labels_test.astype(int)
#         labels_b = np.eye(11,dtype=int)[int_test_labels]
#         batch_xs,batch_ys = get_batches(images_train,labels_a,24,24,16,20)
#         sess.run(train_step1,feed_dict={xs:batch_xs,ys:batch_ys,keep_prob:0.8})
# #         if i<=500:
# #             sess.run(train_step1,feed_dict={xs:images_train,ys:labels_a,keep_prob:0.8})
# #         elif i<=2000:
# #             sess.run(train_step2,feed_dict={xs:images_train,ys:labels_a,keep_prob:0.8})
# #         else:
# #             sess.run(train_step3,feed_dict={xs:images_train,ys:labels_a,keep_prob:0.8})
#         if i%2==0:
#             train_accuracy = compute_accuracy(images_test,labels_b)
#             print('Step {0} tranining accuracy {1}'.format(i,train_accuracy))

# Epoches

config = tf.ConfigProto() 
config.gpu_options.allow_growth = True 

sess = tf.Session()
writer = tf.summary.FileWriter("C:\\Users\\lenovo\\Desktop\\CECS\\4528\\Final_project", sess.graph)
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())
merged = tf.summary.merge_all()
for epoch in range(100):
    for i in range(0, 1904, 16):
        batch_images = images_train[i: i+16]
        batch_labels = labels_a[i: i+16]
    #             data, label = sess.run([batch_images, batch_images])
        iteration,result,_ = sess.run([global_step,scalar_loss,train_step],feed_dict={xs:batch_images,ys:batch_labels,keep_prob:0.8})
    #         test_accuracy = compute_accuracy(images_test,labels_b)
    train_accuracy = compute_accuracy(images_train,labels_a)
    writer.add_summary(result, epoch)
    #         print("Epoch %d,Test accuracy %g" % (epoch, test_accuracy))
    print("Epoch %d,Train accuracy %g" % (epoch, train_accuracy))
test_accuracy = compute_accuracy(images_test,labels_b)